# 03 - LangFuse Evaluations & Prompt Management

## 📖 Mục tiêu học tập

Notebook này sẽ hướng dẫn bạn:
- Hiểu cách thức đánh giá (evaluation) hệ thống LLM với LangFuse
- Thực hiện đánh giá thủ công (human feedback) và tự động
- Quản lý prompt versions thông qua LangFuse Prompt Management
- Theo dõi hiệu suất và cải thiện chất lượng model qua thời gian
- Tích hợp evaluation workflow vào quy trình phát triển LLM

## 🌟 Giới thiệu

### Tầm quan trọng của Evaluation trong LLM Development

Đánh giá là một phần thiết yếu trong phát triển ứng dụng LLM vì:

1. **Đảm bảo chất lượng**: Kiểm tra độ chính xác và phù hợp của output
2. **Theo dõi hiệu suất**: Giám sát sự thay đổi chất lượng theo thời gian
3. **A/B Testing**: So sánh hiệu quả của các prompt/model khác nhau
4. **Compliance & Safety**: Đảm bảo output tuân thủ quy định và an toàn

### LangFuse Evaluation Features

- **Human Feedback**: Thu thập đánh giá từ người dùng thực tế
- **Automated Scoring**: Đánh giá tự động dựa trên rule-based hoặc model-based
- **Prompt Management**: Version control cho prompts với A/B testing
- **Analytics Dashboard**: Visualize evaluation metrics và trends

## ⚙️ Cài đặt & Cấu hình

In [ ]:
# Cài đặt dependencies cần thiết
!pip install langfuse langchain-anthropic python-dotenv uuid

In [ ]:
import os
import json
import uuid
from datetime import datetime
from typing import Dict, List, Optional

from dotenv import load_dotenv
from langfuse import Langfuse
from langfuse.decorators import langfuse_context, observe
from langchain_anthropic import ChatAnthropic
from langchain.schema import HumanMessage, SystemMessage

# Load environment variables
load_dotenv()

print("✅ Đã import các thư viện cần thiết")

In [ ]:
# Khởi tạo LangFuse client
langfuse = Langfuse(
    secret_key=os.getenv("LANGFUSE_SECRET_KEY"),
    public_key=os.getenv("LANGFUSE_PUBLIC_KEY"),
    host=os.getenv("LANGFUSE_HOST", "http://localhost:3000")
)

# Khởi tạo ChatAnthropic
llm = ChatAnthropic(
    model="claude-3-haiku-20240307",
    api_key=os.getenv("ANTHROPIC_API_KEY"),
    temperature=0.1
)

print("🚀 Đã khởi tạo LangFuse và ChatAnthropic")
print(f"📊 LangFuse Host: {os.getenv('LANGFUSE_HOST', 'http://localhost:3000')}")

## 🔍 Ví dụ 1: Ghi nhận đánh giá thủ công (Human Feedback)

Human feedback là cách quan trọng nhất để đánh giá chất lượng output của LLM vì chỉ con người mới có thể đánh giá được context, tone, và appropriateness.

In [ ]:
@observe()
def generate_customer_response(customer_query: str, context: str = "") -> str:
    """Tạo phản hồi cho khách hàng dựa trên query và context"""
    
    system_prompt = """Bạn là một chuyên viên hỗ trợ khách hàng chuyên nghiệp. 
    Hãy trả lời câu hỏi của khách hàng một cách thân thiện, chính xác và hữu ích.
    Nếu có thông tin context, hãy sử dụng để đưa ra câu trả lời phù hợp nhất."""
    
    messages = [
        SystemMessage(content=system_prompt),
        HumanMessage(content=f"Context: {context}\n\nCâu hỏi khách hàng: {customer_query}")
    ]
    
    response = llm.invoke(messages)
    return response.content

In [ ]:
# Test customer service responses
test_queries = [
    {
        "query": "Tôi muốn đổi trả sản phẩm đã mua 3 ngày trước",
        "context": "Chính sách đổi trả: 7 ngày, sản phẩm còn nguyên seal",
        "trace_id": str(uuid.uuid4())
    },
    {
        "query": "Làm sao để theo dõi đơn hàng của tôi?",
        "context": "Hệ thống tracking có sẵn trên website, cần mã đơn hàng",
        "trace_id": str(uuid.uuid4())
    },
    {
        "query": "Sản phẩm này có bảo hành bao lâu?",
        "context": "Điện thoại iPhone: bảo hành 12 tháng chính hãng",
        "trace_id": str(uuid.uuid4())
    }
]

responses = []

for test_case in test_queries:
    # Set trace context
    langfuse_context.configure(
        trace_id=test_case["trace_id"],
        session_id="customer_support_evaluation",
        user_id="evaluator_001"
    )
    
    response = generate_customer_response(
        customer_query=test_case["query"],
        context=test_case["context"]
    )
    
    responses.append({
        "query": test_case["query"],
        "response": response,
        "trace_id": test_case["trace_id"]
    })
    
    print(f"❓ Query: {test_case['query']}")
    print(f"🤖 Response: {response}")
    print(f"🔗 Trace ID: {test_case['trace_id']}")
    print("-" * 80)

In [ ]:
# Simulate human feedback collection
def collect_human_feedback(trace_id: str, response_text: str, query: str):
    """Thu thập feedback từ người đánh giá"""
    
    print(f"\n📋 Đánh giá phản hồi cho: {query}")
    print(f"🤖 Phản hồi: {response_text}")
    print("\n📊 Tiêu chí đánh giá:")
    print("1. Helpfulness (1-5): Mức độ hữu ích")
    print("2. Accuracy (1-5): Độ chính xác")
    print("3. Friendliness (1-5): Độ thân thiện")
    print("4. Overall (1-5): Đánh giá tổng thể")
    
    # Simulate evaluator scores (trong thực tế sẽ input từ UI)
    import random
    scores = {
        "helpfulness": random.randint(3, 5),
        "accuracy": random.randint(3, 5),
        "friendliness": random.randint(3, 5),
        "overall": random.randint(3, 5)
    }
    
    # Tạo comment mẫu
    comments = [
        "Phản hồi rất chuyên nghiệp và đầy đủ thông tin",
        "Cần cải thiện thêm về độ chi tiết",
        "Tone rất phù hợp với khách hàng",
        "Chính xác và hữu ích"
    ]
    
    feedback_comment = random.choice(comments)
    
    return scores, feedback_comment

# Thu thập feedback cho tất cả responses
for response_data in responses:
    scores, comment = collect_human_feedback(
        response_data["trace_id"],
        response_data["response"],
        response_data["query"]
    )
    
    # Ghi feedback vào LangFuse
    langfuse.score(
        trace_id=response_data["trace_id"],
        name="helpfulness",
        value=scores["helpfulness"],
        comment=f"Human feedback: {comment}"
    )
    
    langfuse.score(
        trace_id=response_data["trace_id"],
        name="accuracy",
        value=scores["accuracy"]
    )
    
    langfuse.score(
        trace_id=response_data["trace_id"],
        name="friendliness",
        value=scores["friendliness"]
    )
    
    langfuse.score(
        trace_id=response_data["trace_id"],
        name="overall_rating",
        value=scores["overall"],
        comment=comment
    )
    
    print(f"✅ Đã ghi feedback cho trace: {response_data['trace_id'][:8]}...")
    print(f"📊 Scores: {scores}")
    print(f"💬 Comment: {comment}\n")

print("🎉 Hoàn thành thu thập human feedback!")

## 🤖 Ví dụ 2: Đánh giá tự động (Automated Evaluation)

Automated evaluation cho phép scale việc đánh giá và đảm bảo consistency. Chúng ta sẽ implement các evaluator cho JSON format, safety, và content quality.

In [ ]:
# JSON Format Evaluator
def evaluate_json_format(response_text: str) -> Dict:
    """Kiểm tra xem response có đúng format JSON không"""
    try:
        parsed = json.loads(response_text)
        return {
            "score": 1.0,
            "passed": True,
            "reason": "Valid JSON format",
            "details": {"keys": list(parsed.keys()) if isinstance(parsed, dict) else "array"}
        }
    except json.JSONDecodeError as e:
        return {
            "score": 0.0,
            "passed": False,
            "reason": f"Invalid JSON: {str(e)}",
            "details": {"error": str(e)}
        }

# Safety Evaluator
def evaluate_safety(response_text: str) -> Dict:
    """Kiểm tra content safety (simplified version)"""
    unsafe_keywords = [
        "hack", "phishing", "scam", "illegal", "virus",
        "password", "credit card", "sensitive"
    ]
    
    found_unsafe = []
    for keyword in unsafe_keywords:
        if keyword.lower() in response_text.lower():
            found_unsafe.append(keyword)
    
    if found_unsafe:
        return {
            "score": 0.0,
            "passed": False,
            "reason": f"Unsafe keywords detected: {found_unsafe}",
            "details": {"unsafe_keywords": found_unsafe}
        }
    else:
        return {
            "score": 1.0,
            "passed": True,
            "reason": "No unsafe content detected",
            "details": {"checked_keywords": len(unsafe_keywords)}
        }

# Length Evaluator
def evaluate_response_length(response_text: str, min_length: int = 50, max_length: int = 500) -> Dict:
    """Kiểm tra độ dài phản hồi có phù hợp không"""
    length = len(response_text)
    
    if length < min_length:
        return {
            "score": 0.3,
            "passed": False,
            "reason": f"Response too short: {length} chars (min: {min_length})",
            "details": {"length": length, "min_required": min_length}
        }
    elif length > max_length:
        return {
            "score": 0.7,
            "passed": False,
            "reason": f"Response too long: {length} chars (max: {max_length})",
            "details": {"length": length, "max_allowed": max_length}
        }
    else:
        return {
            "score": 1.0,
            "passed": True,
            "reason": f"Appropriate length: {length} chars",
            "details": {"length": length, "min": min_length, "max": max_length}
        }

print("✅ Đã định nghĩa các automated evaluators")

In [ ]:
@observe()
def generate_product_info(product_name: str, format_type: str = "json") -> str:
    """Tạo thông tin sản phẩm theo format được yêu cầu"""
    
    if format_type == "json":
        system_prompt = """Bạn là một AI assistant chuyên cung cấp thông tin sản phẩm.
        Hãy trả về thông tin sản phẩm dưới dạng JSON với các field: name, price, description, features, availability.
        Chỉ trả về JSON, không có text khác."""
    else:
        system_prompt = """Bạn là một AI assistant chuyên cung cấp thông tin sản phẩm.
        Hãy mô tả sản phẩm một cách chi tiết và hấp dẫn."""
    
    messages = [
        SystemMessage(content=system_prompt),
        HumanMessage(content=f"Cung cấp thông tin về sản phẩm: {product_name}")
    ]
    
    response = llm.invoke(messages)
    return response.content

# Test với các sản phẩm khác nhau
test_products = [
    {"name": "iPhone 15 Pro", "format": "json", "trace_id": str(uuid.uuid4())},
    {"name": "MacBook Air M2", "format": "json", "trace_id": str(uuid.uuid4())},
    {"name": "AirPods Pro", "format": "text", "trace_id": str(uuid.uuid4())}
]

product_responses = []

for product in test_products:
    langfuse_context.configure(
        trace_id=product["trace_id"],
        session_id="product_info_evaluation",
        user_id="system_test"
    )
    
    response = generate_product_info(
        product_name=product["name"],
        format_type=product["format"]
    )
    
    product_responses.append({
        "product": product["name"],
        "format": product["format"],
        "response": response,
        "trace_id": product["trace_id"]
    })
    
    print(f"📱 Product: {product['name']} ({product['format']})")
    print(f"🔗 Trace: {product['trace_id']}")
    print(f"📝 Response: {response[:200]}...")
    print("-" * 80)

## 📝 Ví dụ 3: Prompt Management với LangFuse

Prompt Management cho phép version control prompts, A/B test different versions, và theo dõi performance của từng prompt version.

In [ ]:
# Define các prompt versions cho customer support
customer_support_prompts = {
    "v1_basic": {
        "content": """Bạn là một nhân viên hỗ trợ khách hàng. 
Hãy trả lời câu hỏi của khách hàng một cách lịch sự và hữu ích.

Câu hỏi: {question}
Context: {context}""",
        "labels": ["customer-support", "basic"]
    },
    
    "v2_detailed": {
        "content": """Bạn là một chuyên viên hỗ trợ khách hàng chuyên nghiệp của công ty.
Khi trả lời khách hàng, hãy:
1. Thể hiện sự đồng cảm và hiểu biết
2. Cung cấp thông tin chính xác và chi tiết
3. Đưa ra các bước hành động cụ thể nếu cần
4. Kết thúc bằng việc hỏi thêm nếu khách hàng cần hỗ trợ gì khác

Context có sẵn: {context}
Câu hỏi của khách hàng: {question}

Phản hồi của bạn:""",
        "labels": ["customer-support", "detailed", "structured"]
    },
    
    "v3_empathetic": {
        "content": """Bạn là một chuyên viên hỗ trợ khách hàng giàu kinh nghiệm và rất quan tâm đến trải nghiệm khách hàng.

Nguyên tắc phản hồi:
- Luôn bắt đầu bằng việc thừa nhận và đồng cảm với tình huống của khách hàng
- Sử dụng ngôn ngữ ấm áp, thân thiện nhưng vẫn chuyên nghiệp
- Giải thích rõ ràng các policy và quy trình
- Đưa ra các lựa chọn và giải pháp thay thế nếu có thể
- Đảm bảo khách hàng cảm thấy được lắng nghe và quan tâm

Thông tin tham khảo: {context}
Câu hỏi/Vấn đề của khách hàng: {question}

Hãy trả lời một cách chu đáo và hỗ trợ tốt nhất:""",
        "labels": ["customer-support", "empathetic", "advanced"]
    }
}

print(f"📝 Defined {len(customer_support_prompts)} prompt versions")

In [ ]:
@observe()
def respond_with_prompt_version(question: str, context: str, prompt_version: str):
    """Sử dụng specific prompt version để trả lời"""
    
    # Get prompt content
    prompt_content = customer_support_prompts[prompt_version]["content"]
    
    # Format prompt với variables
    formatted_prompt = prompt_content.format(
        question=question,
        context=context
    )
    
    # Gọi LLM
    messages = [HumanMessage(content=formatted_prompt)]
    response = llm.invoke(messages)
    
    # Log prompt version được sử dụng
    langfuse_context.update_current_trace(
        metadata={
            "prompt_version": prompt_version,
            "prompt_name": "customer_support_response"
        }
    )
    
    return response.content

print("✅ Đã setup prompt management functions")

In [ ]:
# A/B Test các prompt versions
test_scenarios = [
    {
        "question": "Tôi đặt hàng 1 tuần rồi mà chưa nhận được, rất thất vọng!",
        "context": "Đơn hàng #12345, ship date dự kiến: 3-5 ngày, hiện tại đã 7 ngày",
        "scenario_id": "delayed_delivery"
    },
    {
        "question": "Sản phẩm tôi nhận được bị lỗi, muốn đổi trả",
        "context": "Chính sách: đổi trả trong 7 ngày, cần hóa đơn và sản phẩm nguyên vẹn",
        "scenario_id": "defective_product"
    }
]

# Test từng prompt version với mỗi scenario
comparison_results = []

for scenario in test_scenarios:
    print(f"\n🧪 Testing scenario: {scenario['scenario_id']}")
    print(f"❓ Question: {scenario['question']}")
    
    scenario_results = {
        "scenario_id": scenario["scenario_id"],
        "question": scenario["question"],
        "responses": {}
    }
    
    for version in ["v1_basic", "v2_detailed", "v3_empathetic"]:
        trace_id = str(uuid.uuid4())
        
        langfuse_context.configure(
            trace_id=trace_id,
            session_id=f"prompt_ab_test_{scenario['scenario_id']}",
            user_id="prompt_tester",
            tags=["ab_test", f"prompt_{version}", scenario["scenario_id"]]
        )
        
        response = respond_with_prompt_version(
            question=scenario["question"],
            context=scenario["context"],
            prompt_version=version
        )
        
        scenario_results["responses"][version] = {
            "response": response,
            "trace_id": trace_id,
            "length": len(response)
        }
        
        print(f"\n📝 {version.upper()}:")
        print(f"Response ({len(response)} chars): {response[:150]}...")
        print(f"Trace: {trace_id}")
    
    comparison_results.append(scenario_results)
    print("-" * 80)

print("\n🎯 Hoàn thành A/B testing các prompt versions!")

## 📈 Giải thích & Phân tích

### Truy cập LangFuse Dashboard

Để xem và phân tích kết quả evaluation:

1. **Mở LangFuse UI**: Truy cập `http://localhost:3000` (hoặc URL của LangFuse instance)

2. **Xem Traces**: 
   - Navigate đến "Traces" tab
   - Filter theo session_id để xem specific evaluation runs
   - Click vào individual traces để xem chi tiết

3. **Phân tích Scores**:
   - Trong "Scores" tab, xem distribution của các evaluation metrics
   - So sánh performance giữa các prompt versions
   - Track trends theo thời gian

4. **Prompt Management**:
   - Trong "Prompts" tab, xem các versions đã tạo
   - Compare performance của different versions
   - Deploy prompts tốt nhất vào production

In [ ]:
# Flush all data to LangFuse
langfuse.flush()
print("✅ All evaluation data has been sent to LangFuse!")
print("📊 Check LangFuse Dashboard at http://localhost:3000 for detailed analysis")

## 🎯 Kết luận & Bước tiếp theo

### Những gì đã học được

1. **Human Feedback Collection**: Cách thu thập và ghi nhận feedback từ người dùng thực tế
2. **Automated Evaluation**: Implement các evaluator tự động cho format, safety, và quality
3. **Prompt Management**: Version control và A/B testing cho prompts
4. **Performance Analysis**: Phân tích và so sánh hiệu suất của các approaches khác nhau
5. **LangFuse Integration**: Sử dụng LangFuse để tracking và visualization

### Best Practices đã áp dụng

- **Structured Evaluation**: Định nghĩa rõ các tiêu chí đánh giá
- **Automated + Human**: Kết hợp automation với human judgment
- **Version Control**: Theo dõi changes và performance của prompts
- **Continuous Monitoring**: Setup pipeline để evaluation liên tục

### Bước tiếp theo

1. **Production Deployment**: 
   - Deploy prompt version tốt nhất vào production
   - Implement gradual rollout strategy

2. **Automated Pipeline**:
   - Setup CI/CD cho prompt testing
   - Automated regression testing

3. **Advanced Evaluation**:
   - Model-based evaluation (using LLM as judge)
   - Multi-dimensional scoring
   - Real-time feedback collection

4. **Scaling**:
   - Evaluation cho multiple use cases
   - Cross-model comparison
   - Performance optimization

### Resources cho học thêm

- [LangFuse Documentation](https://langfuse.com/docs)
- [LLM Evaluation Best Practices](https://langfuse.com/guides/evaluation)
- [Prompt Engineering Guide](https://www.promptingguide.ai/)
- [A/B Testing for AI Systems](https://langfuse.com/guides/ab-testing)